In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

```bash
conda create -n edu4 python=3.11 jupyter matplotlib
```

```bash 
! pip install -U -r requirements.txt
```

```bash
! pip install -U numpy
! pip install -U scikit-learn
```

## Update repository

In [ ]:
# ! git pull

## Add import path

In [ ]:
import os
import sys
import gc

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
del module_path

## Organize imports

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from huggingface_hub import hf_hub_download, notebook_login
import numpy as np
import torch
from torch import nn

In [ ]:
from sae_lens import SAE, HookedSAETransformer
from transformer_lens.utils import tokenize_and_concatenate

In [ ]:
from transformer_lens import HookedTransformer

In [ ]:
from datasets import load_dataset

In [ ]:
import multiprocessing
from pathlib import Path

In [ ]:
from tqdm import tqdm

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
import plotly.express as px

In [ ]:
from src.lattmc.fca.utils import *
from src.lattmc.fca.data_utils import *
from src.lattmc.fca.image_utils import *
from src.lattmc.fca.models import *
from src.lattmc.fca.fca_utils import *
from src.lattmc.fca.image_gens import *

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2024

In [ ]:
# For the most part I'll try to import functions and classes near where they are used
# to make it clear where they come from.

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

In [ ]:
torch.__version__

In [ ]:
np.__version__

## Initialize Path

In [ ]:
PATH = Path('../data')
checkpoint_dir = PATH / 'saes'
dataset_suffix = 'wiki'
vectors_name = f'gpt2_small_10_{dataset_suffix}'
checkpoint_dir.mkdir(exist_ok=True, parents=True)
vectors_dir = checkpoint_dir / f'{vectors_name}_10_vecs'
vectors_dir.mkdir(exist_ok=True, parents=True)
matrix_dir = checkpoint_dir / f'{vectors_name}_10_mats'
matrix_dir.mkdir(exist_ok=True, parents=True)
vectors_path = checkpoint_dir / f'{vectors_name}_10_vecs.joblib'

image_dir = PATH / 'images'
image_path = image_dir / '1024.png'

## Initialize simple dataset

In [ ]:
dataset = load_dataset(
    path=f'NeelNanda/{dataset_suffix}-10k',
    split='train',
    streaming=False,
)

## Initialize model

In [ ]:
class Text2Latent(object):

    def __init__(self, model: nn.Module, sae: nn.Module):
        self.model = model.eval()
        self.sae = sae.eval()
        self.hook_point = sae.cfg.hook_name

    @torch.inference_mode()
    def tokenize(self, text):
        return self.model.to_tokens(text)

    @torch.inference_mode()
    def to_string(self, tokens):
        return self.model.to_string(tokens)

    @torch.inference_mode()
    def encode(self, text):
        _, cache = self.model.run_with_cache(text, prepend_bos=True)
        # get the feature activations from our SAE
        z = self.sae.encode(cache[self.hook_point])
        
        return z

    @torch.inference_mode()
    def decode(self, z):
        return self.sae.decode(z)

    @torch.inference_mode()
    def forward(self, text):
        z = self.encode(text)
        r = self.decode(z)

        return r

In [ ]:
def gen_concept(idx, val, shape=24576):
    v_idx = np.zeros((shape,), dtype=float)
    v_idx[idx] = val
    concept = fca.G_FG(v_idx)
    S = dsort(V[concept.A], idx)
    
    return v_idx, concept, concept.A[S]

In [ ]:
layer = 10

In [ ]:
model_name = 'gpt2-small'
release = 'gpt2-small-res-jb'
sae_id = f'blocks.{layer}.hook_resid_pre'
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release=release,  # see other options in sae_lens/pretrained_saes.yaml
    sae_id=sae_id,  # won't always be a hook point
    device=device,
)
hook_point = sae.cfg.hook_name
print(hook_point)

In [ ]:
model = HookedTransformer.from_pretrained(model_name, device=device)

In [ ]:
net = Text2Latent(model, sae)

## Generate V Lattice

In [ ]:
gc.collect()

In [ ]:
if any(Path(matrix_dir).iterdir()):
    print(f'{matrix_dir} is not empty')
else:
    with tqdm(dataset) as pdata:
        for idx, d in enumerate(pdata):
            t = d['text']
            v = net.encode(t)
            v_sparse = csr_matrix(v.to('cpu').detach().numpy()[0])
            joblib.dump(
                v_sparse, 
                matrix_dir / f'{idx}.joblib'
            )

In [ ]:
gc.collect()

In [ ]:
if vectors_path.exists():
    V = joblib.load(vectors_path)
    print(f'Vectors are loaded from {vectors_path}')
else:
    v_paths = list(matrix_dir.glob('*.joblib'))
    error_paths = []
    V_dict = {}
    V_list = []
    with tqdm(v_paths) as v_ppaths:
        for v_path in v_ppaths:
            v_sparse = joblib.load(v_path)
            vs = v_sparse.toarray()[1:]
            v = np.maximum.reduce(vs)
            V_dict[int(v_path.stem)] = v
    with tqdm(list(range(10000))) as prange:
        for k in prange:
            V_list.append(V_dict[k])
    V = np.array(V_list)
    joblib.dump(V, vectors_path)    

## Generate Context and Analyze

In [ ]:
gc.collect()

In [ ]:
fca = FCA(V)

In [ ]:
text1 = "The Golden Gate Bridge"
z = net.encode(text1)
zs = z.to('cpu').detach().numpy()[0]
v = np.maximum.reduce(zs)

In [ ]:
zs[1:].shape

In [ ]:
zs.shape

In [ ]:
v[19837]

## Detect Negation

In [ ]:
w1 = 'Bird'
w2 = 'can'
w3 = 'fly'
w4 = 'hunt'

In [ ]:
ws = [w1, w2, w3, w4]

In [ ]:
text = ' '.join(ws)
text

In [ ]:
vs = net.encode(text)

In [ ]:
ts = net.tokenize(text)

In [ ]:
ts.shape

In [ ]:
net.to_string(ts[0][4])

In [ ]:
vs.shape

In [ ]:
ts = net.tokenize(w1)[0]
net.to_string(ts[1])

In [ ]:
ts = net.tokenize(w2)[0]
net.to_string(ts[1])

In [ ]:
ts = net.tokenize(w3)[0]
net.to_string(ts[1])

In [ ]:
ts = net.tokenize(w4)[0]
net.to_string(ts[1])

In [ ]:
v1 = net.encode(w1)[0][1] / 10
v2 = net.encode(w2)[0][1] / 10
v3 = net.encode(w3)[0][1] / 10
v4 = net.encode(w4)[0][1] / 10
v1.shape

In [ ]:
to_numpy(v1)[np.argmax(to_numpy(v1))]

In [ ]:
to_numpy(v2)[np.argmax(to_numpy(v2))]

In [ ]:
to_numpy(v3)[np.argmax(to_numpy(v3))]

In [ ]:
to_numpy(v4)[np.argmax(to_numpy(v4))]

In [ ]:
vt = vs[0][1:]

In [ ]:
vt.shape

In [ ]:
c1 = fca.G_FG(v1)
c2 = fca.G_FG(v2)
c3 = fca.G_FG(v3)
c4 = fca.G_FG(v4)

In [ ]:
c3

In [ ]:
dataset[c3.A]

In [ ]:
c1 & c2

## Landmark Detection?

In [ ]:
text = "Golden Gate Bridge"
z = net.encode(text)
zs = z.to('cpu').detach().numpy()[0]

In [ ]:
z[0].shape

In [ ]:
l = meet(zs[1], zs[2])

In [ ]:
topK(l, 20)

In [ ]:
concept = fca.G_FG(l)
concept

In [ ]:
dataset[concept.A[0].item()]

## Landmark Detection?

In [ ]:
import numpy as np
a = np.array([[9,2,3],[4,5,6],[7,0,5]])
col_index = 0
sorted_indices = np.argsort(a[:, col_index])[::-1]
sorted_array = a[sorted_indices]
print("Sorted array:", sorted_array)
print("Original indices:", sorted_indices)

In [ ]:
np.argsort(

In [ ]:
text = "Golden Gate Bridge is an iconic landmark in San Francisco"
z = net.encode(text)
zs = z.to('cpu').detach().numpy()[0]

In [ ]:
z[0].shape

In [ ]:
tokens = net.tokenize(text)[0]
words = [net.to_string(t) for t in tokens]
tokens, words

In [ ]:
l = meet(zs[1], zs[2])

In [ ]:
topK(l, 20)

In [ ]:
concept = fca.G_FG(l)
concept

In [ ]:
S = dsort(V[concept.A], 21286)

In [ ]:
dataset[S[0].item()]

## Species Detection?

In [ ]:
text = "Golden Retriever"
z = net.encode(text)
zs = z.to('cpu').detach().numpy()[0]

In [ ]:
tokens = net.tokenize(text)
tokens

In [ ]:
tokens[0][2:]

In [ ]:
net.to_string(4990)

In [ ]:
z[0].shape

In [ ]:
zs.shape

In [ ]:
l = join_all(zs[2:])

In [ ]:
l = meet(zs[2], l)

In [ ]:
topK(l, 20)

In [ ]:
concept = fca.G_FG(l / 8)
concept

In [ ]:
S = dsort(V[concept.A], 6950)

In [ ]:
dataset[S[0].item()]

In [ ]:
v_test, concept_test, S = gen_concept(12610, 12)
concept_test

In [ ]:
dataset[S[0].item()]

## Golden Feature from Neuronscope

In [ ]:
v_golden, concept, S = gen_concept(19837, 90)
v_golden.shape, concept

In [ ]:
concept.A, S

In [ ]:
for v_s in V[S]:
    print(v_s[19837])

In [ ]:
V[S][:, 19837]

In [ ]:
dataset[S[0].item()]

#### Find common features

In [ ]:
tk = topK(concept.v, 40)
tk

In [ ]:
idx = 5
args = tk[1][idx], tk[0][idx]
v_test, concept_test = gen_concept(*args)
print(f'{args}, {v_test.shape}, {concept_test}')
dataset[concept_test.A[0].item()]

## Cat Feature from Neuronscope

In [ ]:
v_cat, concept = gen_concept(16899, 80)
v_cat.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Apple Feature from Neuronscope

In [ ]:
v_apple, concept = gen_concept(4269, 70)
v_apple.shape, concept

In [ ]:
dataset[concept.A[0].item()]

#### Find common features

In [ ]:
idx = 34
tk = topK(concept.v, 40)
args = tk[1][idx], tk[0][idx]
v_test, concept_test = gen_concept(*args)
print(f'{args}, {v_test.shape}, {concept_test}')
dataset[concept_test.A[0].item()]

## Thunder and Lightning Feature from Neuronscope

In [ ]:
gc.collect()

In [ ]:
v_thunder, concept = gen_concept(23123, 20)
v_thunder.shape, concept

In [ ]:
dataset[concept.A[0].item()]

In [ ]:
idx = 4
tk = topK(concept.v, 40)
args = tk[1][idx], tk[0][idx]
v_test, concept_test = gen_concept(*args)
print(f'{args}, {v_test.shape}, {concept_test}')
dataset[concept_test.A[0].item()]

## School and Lightning Feature from Neuronscope

In [ ]:
v_school, concept = gen_concept(20781, 84)
v_school.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## King and Lightning Feature from Neuronscope

In [ ]:
v_king, concept = gen_concept(17624, 60)
v_king.shape, concept

In [ ]:
dataset[concept.A[1].item()]

In [ ]:
idx = 3
tk = topK(concept.v, 40)
args = tk[1][idx], tk[0][idx] + 20
v_test, concept_test = gen_concept(*args)
print(f'{args}, {v_test.shape}, {concept_test}')
dataset[concept_test.A[0].item()]

## Orgs and Lightning Feature from Neuronscope

In [ ]:
v_orgs, concept = gen_concept(16660, 60)
v_orgs.shape, concept

In [ ]:
dataset[concept.A[0].item()]

In [ ]:
idx = 2
tk = topK(concept.v, 40)
args = tk[1][idx], tk[0][idx] + 8
v_test, concept_test = gen_concept(*args)
print(f'{args}, {v_test.shape}, {concept_test}')
dataset[concept_test.A[0].item()]

## States and Lightning Feature from Neuronscope

In [ ]:
v_state, concept = gen_concept(22, 10)
v_state.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Joint concepts

In [ ]:
idx = 1
tk = topK(concept.v, 40)
args = tk[1][idx], tk[0][idx] + 4
v_test, concept_test = gen_concept(*args)
print(f'{args}, {v_test.shape}, {concept_test}')
dataset[concept_test.A[0].item()]

In [ ]:
# topK(v_orgs, 4), topK(v_school, 4)

In [ ]:
v_orgs = np.zeros_like(to_numpy(v))
v_school = np.zeros_like(to_numpy(v))
v_having = np.zeros_like(to_numpy(v))

In [ ]:
v_orgs[16660] = 20
v_school[17624] = 20
v_having[17935] = 6

In [ ]:
orgs = fca.G_FG(v_orgs)
schools = fca.G_FG(v_school)
havings = fca.G_FG(v_having)
orgs, schools, havings

In [ ]:
org_school = orgs & schools
org_school

In [ ]:
dataset[org_school.A[0].item()]['text']

In [ ]:
org_school_havings = orgs & schools & havings 
org_school_havings

In [ ]:
dataset[org_school_havings.A[0].item()]['text']

## Recipie Feature from Neuronscope

In [ ]:
v_recipie, concept = gen_concept(7, 28)
v_recipie.shape, concept

In [ ]:
dataset[concept.A[0].item()]

In [ ]:
idx = 1
tk = topK(concept.v, 40)
args = tk[1][idx], tk[0][idx]
v_test, concept_test = gen_concept(*args)
print(f'{args}, {v_test.shape}, {concept_test}')
dataset[concept_test.A[0].item()]

## Election Feature from Neuronscope

In [ ]:
v_elect, concept = gen_concept(29, 28)
v_elect.shape, concept

In [ ]:
dataset[concept.A[0].item()]

In [ ]:
idx = 2
tk = topK(concept.v, 40)
args = tk[1][idx], tk[0][idx]
v_test, concept_test = gen_concept(*args)
print(f'{args}, {v_test.shape}, {concept_test}')
dataset[concept_test.A[0].item()]

## Mixture of Election and Party

In [ ]:
v_elect, concept_elects = gen_concept(29, 2)
v_elect.shape, concept

## Food Features

In [ ]:
texts = [
    'food recipie',
    'love',
    'admire',
    'sex',
]

In [ ]:
v = net.encode(texts[2])[0][1]
concept = fca.G_FG(v / 4)
concept

In [ ]:
dataset[concept.A[3].item()]

In [ ]:
topK(v, 10)

In [ ]:
v_love, recipie = gen_concept(14654, 20)
v_love.shape, concept

In [ ]:
v_admire, recipie = gen_concept(14990, 20)
v_admire.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Verbal Feature from Neuronscope

In [ ]:
v_verbal, concept = gen_concept(33, 40)
v_verbal.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Border Feature from Neuronscope

In [ ]:
v_border, concept = gen_concept(35, 20)
v_border.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Cross Concept

In [ ]:
v_verbal, concept_verbal = gen_concept(33, 16)
v_border, concept_border = gen_concept(35, 16)

In [ ]:
cross = concept_verbal & concept_border
cross

In [ ]:
dataset[cross.A[0].item()]

## Cross Concept Probability

In [ ]:
v_posib, concept_posib = gen_concept(1061, 20)
v_posit, concept_posit = gen_concept(809, 20)
v_liklh, concept_liklh = gen_concept(418, 20)
v_qualt, concept_qualt = gen_concept(129, 20)

In [ ]:
cross_pr_ps = concept_posib & concept_posit
cross_pr_ps

In [ ]:
dataset[cross_pr_ps.A[0].item()]

In [ ]:
cross_lk_ql = concept_liklh & concept_qualt
cross_lk_ql

In [ ]:
dataset[cross_lk_ql.A[0].item()]

## Context Features

In [ ]:
v_peval, concept_peval = gen_concept(3109, 20)
v_evstt, concept_evstt = gen_concept(3784, 20)
v_sosis, concept_sosis = gen_concept(3702, 20)
v_agevs, concept_agevs = gen_concept(3388, 20)

In [ ]:
concept_pe_ev = concept_peval & concept_evstt
concept_pe_ev

In [ ]:
dataset[concept_pe_ev.A[0].item()]

In [ ]:
concept_so_ag = concept_sosis & concept_agevs
concept_so_ag

In [ ]:
dataset[concept_so_ag.A[0].item()]

In [ ]:
concept_all = concept_peval & concept_evstt & concept_sosis & concept_agevs
concept_all

In [ ]:
dataset[concept_all.A[0].item()]

## Computer and Technilogies Concepts

In [ ]:
v_comps, concept_comps = gen_concept(20542, 14)
v_comps.shape, concept_comps

In [ ]:
dataset[concept_comps.A[0].item()]

## Innovation Features

In [ ]:
v_innov, concept_innov = gen_concept(2503, 10)
v_innov.shape, concept_innov

In [ ]:
dataset[concept_innov.A[0].item()]

In [ ]:
concept_co_in = concept_comps & concept_innov
concept_co_in

In [ ]:
dataset[concept_co_in.A[0].item()]

## Add Examples

In [ ]:
text1 = " The Golden Gate Bridge"
z = net.encode(text1)
tokens = net.tokenize(text1)
print(torch.topk(z, 12))
print()
print(z)

In [ ]:
r = net.encode('Golden')
torch.topk(r[0][1], 50)

In [ ]:
torch.nonzero(r[0][1]).shape

In [ ]:
texts = [
    "Golden Gate Bridge",
    "New York City",
    "Silicon Valley",
    "The White House",
    "Apple Inc."
]

In [ ]:
texts = [
    "School of AI",
    "School of Economics",
    "School of Medicine",
    "School of Arts",
    "School of Technologies"
]

In [ ]:
texts = "Golden Retriever"
tokens = net.tokenize(texts)
tokens, net.to_string(tokens)

In [ ]:
texts = "Gold"
tokens = net.tokenize(texts)
vs = net.encode(texts)
tokens, net.to_string(tokens)

In [ ]:
net.to_string(32378)

In [ ]:
vs = net.encode(texts)

In [ ]:
vs.shape, vs[0].shape

In [ ]:
v_t = [v[1] for v in vs[:3]]
v_t[0].shape

In [ ]:
v_A = v_t[0]
for v in v_t:
    v_A = torch.minimum(v_A, v)

In [ ]:
vs[0][0].shape, 19837

In [ ]:
torch.topk(vs[0][1], 20)

In [ ]:
torch.topk(vs[0][1], 20)

In [ ]:
torch.topk(vs[0][1], 20)

In [ ]:
torch.topk(v_A, 30)

## Golden Second Index

In [ ]:
v_1, concept_1 = gen_concept(21286, 16)
v_1.shape, concept_1

In [ ]:
dataset[concept_1.A[0].item()]

## Gold Second Index

In [ ]:
v_2, concept_2 = gen_concept(9572, 30)
v_2.shape, concept_2

In [ ]:
dataset[concept_2.A[0].item()]

In [ ]:
concept_m = concept_1 & concept_2
concept_m

In [ ]:
dataset[concept_m.A[1].item()]

In [ ]:
tk = topK(V[0], 30)
tk

In [ ]:
v_A = np.zeros_like(v_t[0].to('cpu').detach().numpy())
v_A[17943] = 62
v_A[11811] = 6
v_A[15823] = 4
v_A[4507] = 4
v_A[20161] = 4
concept = fca.G_FG(v_A)
concept

In [ ]:
concept.A.shape, V.shape, V.shape[0] - concept.A.shape[0]

In [ ]:
concept.A[0]

In [ ]:
dataset[concept.A[2].item()]

## Analyze Tokens

In [ ]:
len(dataset)

In [ ]:
checks = []
with tqdm(list(range(len(dataset)))) as pdata:
    for idx in pdata:
        tokens = net.tokenize(dataset[idx]['text'])[0]
        vs = joblib.load(matrix_dir / f'{idx}.joblib')
        checks.append(tokens.shape[0] == vs.shape[0])

In [ ]:
np.all(checks)

In [ ]:
from IPython.display import clear_output
v_paths = list(matrix_dir.glob('*.joblib'))
T_dict = {}
W_dict = {}
with tqdm(concept.A) as v_ppaths:
    for idx in v_ppaths:
        vs = joblib.load(matrix_dir / f'{idx}.joblib').toarray()
        G_x = find_G_x(vs, v_A)
        if G_x.shape[0] > 0:
            T_dict[idx.item()] = G_x
        clear_output(wait=True)

In [ ]:
len(T_dict)

In [ ]:
T_dict[5]

In [ ]:
tokens = net.tokenize(dataset[5]['text'])

In [ ]:
net.to_string(tokens[0][100])

In [ ]:
T_dict[5]

In [ ]:
net.to_string(tokens[0][0]), net.to_string(tokens[0][T_dict[5]])

In [ ]:
W_dict = {}
for k, v in T_dict.items():
    tokens = net.tokenize(dataset[k]['text'])[0]
    W_dict[k] = net.to_string(tokens[v])

In [ ]:
W_dict